# Packages

In [1]:
import pandas as pd
import shapely
import geopandas as gpd

# Input Data

In [2]:
data = pd.read_csv('Input_data/input_data_points.csv')
data.head()

,id,x,y
0,1,480630.212,4202710.876
1,2,480644.768,4202705.689
2,3,480662.238,4202700.740
3,4,480677.958,4202695.060
4,5,480698.530,4202687.650


# Creating a geodataframe of points

In [3]:
geometry = [shapely.geometry.Point(xy) for xy in zip(data.x, data.y)]
geo_df = gpd.GeoDataFrame(data, geometry=geometry)
geo_df.head()

,id,x,y,geometry
0,1,480630.212,4202710.876,POINT (480630.212 4202710.876)
1,2,480644.768,4202705.689,POINT (480644.768 4202705.689)
2,3,480662.238,4202700.740,POINT (480662.238 4202700.740)
3,4,480677.958,4202695.060,POINT (480677.958 4202695.060)
4,5,480698.530,4202687.650,POINT (480698.530 4202687.650)


# Creating a geodataframe of Lines

In [4]:
lines_array = []
for i in range(1,geo_df.shape[0]):
    point_line = shapely.geometry.LineString([geo_df.geometry[i-1], geo_df.geometry[i]])
    line_gdf = gpd.GeoDataFrame({'geometry':[point_line]}, geometry='geometry')
    lines_array.append(line_gdf)

lines_concat = pd.concat(lines_array,)
lines_concat.head()

,geometry
0,"LINESTRING (480630.212 4202710.876, 480644.768..."
0,"LINESTRING (480644.768 4202705.689, 480662.238..."
0,"LINESTRING (480662.238 4202700.740, 480677.958..."
0,"LINESTRING (480677.958 4202695.060, 480698.530..."
0,"LINESTRING (480698.530 4202687.650, 480712.704..."


# Creating dxf file

In [7]:
import ezdxf

# Defining file and model space
doc = ezdxf.new(setup=True)
msp = doc.modelspace()

# Creating 'lines' layer 
lines = doc.layers.add("lines")
lines.color = 3 # green 

# Creating 'texts' layer
texts_points = doc.layers.add("texts_points")
texts_points.color = 4 # cyan

# Drawing lines
for i in range(lines_concat.shape[0]):
        msp.add_lwpolyline(lines_concat.iloc[i].geometry.coords, dxfattribs={"layer": "lines"})

# Drawing texts for points
for i in range(geo_df.shape[0]):
    x0 = geo_df.iloc[i].x
    y0 = geo_df.iloc[i].y
    msp.add_text(str(geo_df.iloc[i].id), dxfattribs={'style': 'LiberationSerif', 'height': 4,
                    "layer": "texts_points"}).set_pos((x0, y0), align='MIDDLE_CENTER')

doc.saveas("Lines_Drawing.dxf")